<a href="https://colab.research.google.com/github/kayjew/ENPH353_Lab2/blob/main/ENPH353_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
from google.colab.patches import cv2_imshow

from google.colab import drive
drive.mount('/content/drive')

video_path = "/content/drive/MyDrive/ENPH353_Lab2/raw_video_feed.mp4"

In [ ]:
cap = cv2.VideoCapture(video_path)

#Get the dimenssions of the image
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
prev_center_pt = (0, 0)
radius = 25
color = (255, 0, 255)
line_thickness = -1

#Check if the video was opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
else:
    print("Video file opened successfully!")

    #Check if frames can be read
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video or error reading frame.")
            break

        #Find the contours of the path
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        thresh = 80
        _, img_bin = cv2.threshold(gray, thresh, 255, cv2.THRESH_BINARY_INV)
        contours, hierarchy = cv2.findContours(img_bin, cv2.RETR_TREE,
                                               cv2.CHAIN_APPROX_SIMPLE)

        #If there are no contours, continue to the next iteration
        if len(contours) == 0:
          cv2.circle(frame, prev_center_pt, radius, color, line_thickness)
          plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
          plt.axis('off')
          plt.show()
          continue

        #Find the lowest y contour
        contour_maxy = 0

        for contour in contours:
            maxy = np.max(contour[:, 0, 1])
            if maxy > contour_maxy:
                contour_maxy = maxy
                lowest_contour = contour

        #Find the x-coordinates of the lowest y contour
        bottom_line = lowest_contour[lowest_contour[:, 0, 1] == contour_maxy]
        min_x = np.min(bottom_line[:, 0, 0])
        max_x = np.max((bottom_line[:, 0, 0]))

        #Draw circle
        center_pt = (int((max_x+min_x)/2), int(contour_maxy)) #(x, y)
        cv2.circle(frame, center_pt, radius, color, line_thickness)
        prev_center_pt = center_pt

        #Display the frame
        cv2_imshow(frame)


# Release the video capture object
cap.release()

"""
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


cv2_imshow(img)

contour_color = (0, 255, 0)
contour_thick = 2
with_contours = cv2.drawContours(frame, contours, -1, contour_color,
                                 contour_thick)
plt.imshow(cv2.cvtColor(with_contours,cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
thresh = 127
ret, img_bin = cv2.threshold(img_gray, thresh, 255, 0)

plt.figure(num=None, figsize=(8, 6), dpi=80)
plt.subplot(1, 2, 1)
plt.imshow(img, cmap="gray")

contours, hierarchy = cv2.findContours(img_bin, cv2.RETR_TREE,
                                       cv2.CHAIN_APPROX_SIMPLE)
contour_color = (0, 255, 0)
contour_thick = 10
with_contours = cv2.drawContours(img, contours, -1, contour_color,
                                 contour_thick)
plt.subplot(1, 2, 2)
plt.imshow(with_contours)
print("{}\n{}".format(type(contours), contours[2].shape))
"""